<a href="https://colab.research.google.com/github/boxiXia/MECE4606-Food-printing/blob/master/Food_printing_starter_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Food printing starter code

# reference for the syringe tip
![alt text](https://images-na.ssl-images-amazon.com/images/I/6192oSnVBxL._SL1006_.jpg "syringe tip reference")
[reference](https://www.amazon.com/dp/B01I1APLIG/?coliid=I1H323R2GGYV4D&colid=1VE48W9F0U468&psc=0&ref_=lv_ov_lig_dp_it)



In [14]:
!pwd

/content


In [0]:
import numpy as np
from numpy.linalg import norm

In [0]:
D = 15.8 # [mm] inner diameter of the cylinder (syringe)
d = 1.64 # [mm] inner diameter of the syring tip, guage 14

TOOL = { # ratio between extruders
    0:[1.0,0.0], # left extruder
    1:[0.0,1.0], # right extruder
    2:[0.5,0.5] # two extruder mixing, for example
}



gcode_para = { # G code parameters
    'D':D, # mm, diameter of the cylinder
    'd':d, # mm, diameter of the extruder
    'applied_percentage':1, # A ratio factor that controls the extruding ratio.
    'TOOL':TOOL,
    'feedrate_move':50,
    'feedrate_quickmove':1000,
    'feedrate_extrude':(d**2) / (330**2),
    }

In [0]:
gcode_initial = ''.join([
    'G92 Z0;set current z as z=0\n',
    'F{0} set feedrate\n'.format(gcode_para['feedrate_move']),
    'G91; Set to Relative Positioning \n'])

In [4]:
print(gcode_initial)

G92 Z0;set current z as z=0
F50 set feedrate
G91; Set to Relative Positioning 



In [0]:
def ExtruderRatio(gcode_para):
    """A function that calculates the extruder ratio. applied_ratio
    = (D/d)^2*applied_percentage
    @boxiXia;@JacobJ77
    """
    D,d,applied_percentage = gcode_para['D'],gcode_para['d'],gcode_para['applied_percentage']    
    ratio = d ** 2 / (D ** 2) # extruder ratio
    #applied_percentage: applied percentage for extruding
    applied_ratio = ratio * applied_percentage
    return applied_ratio

In [0]:
extruder_ratio = ExtruderRatio(gcode_para)

In [7]:
extruder_ratio

0.010773914436788974

In [0]:
path = np.array([
    [0.5,0.5,0],
    [-0.5,-0.5,0],
    [0.5,0.5,0],
    [-0.5,-0.5,0],
    [0.5,0.5,0],
    [1,0,0],
    [0,1,0],
    [-1,0,0],
    [0,-1,0],
])*50 # x,y,z relative movement

e_path = norm(path,axis=1)*extruder_ratio # extruder path

In [0]:
def PrintPath(path,e_path,gcode_para,tool_id = 0):
    
    mix_ratio = gcode_para['TOOL'][tool_id]
    gcode_list = [None for k in range(len(path))]
    for i in range(len(path)):
        gcode_list[i] = 'G1 X{0:0.5f} Y{1:0.5f} Z{2:0.5f} E{3:0.5f}:{4:.5f}:{5:.5f}\n'.\
                format(path[i,0],
                       path[i,1],
                       path[i,2],
                       e_path[i] * mix_ratio[0],
                       e_path[i] * mix_ratio[1],
                       gcode_para['feedrate_move'])
    gcode_list.insert(0,'T2;set tool\n')
    return ''.join(gcode_list)

In [0]:
g_code_path = PrintPath(path,e_path,gcode_para)

In [11]:
print(g_code_path)

T2;set tool
G1 X25.00000 Y25.00000 Z0.00000 E0.38092:0.00000:50.00000
G1 X-25.00000 Y-25.00000 Z0.00000 E0.38092:0.00000:50.00000
G1 X25.00000 Y25.00000 Z0.00000 E0.38092:0.00000:50.00000
G1 X-25.00000 Y-25.00000 Z0.00000 E0.38092:0.00000:50.00000
G1 X25.00000 Y25.00000 Z0.00000 E0.38092:0.00000:50.00000
G1 X50.00000 Y0.00000 Z0.00000 E0.53870:0.00000:50.00000
G1 X0.00000 Y50.00000 Z0.00000 E0.53870:0.00000:50.00000
G1 X-50.00000 Y0.00000 Z0.00000 E0.53870:0.00000:50.00000
G1 X0.00000 Y-50.00000 Z0.00000 E0.53870:0.00000:50.00000



In [0]:
g_code_end = "G0 X-10 Y-10 Z5\nG90; Set to Absolute Positioning\n"

In [13]:
s = gcode_initial+g_code_path+g_code_end
f = open('square.g','w')
f.write(s)
f.close()
print(s)

G92 Z0;set current z as z=0
F50 set feedrate
G91; Set to Relative Positioning 
T2;set tool
G1 X25.00000 Y25.00000 Z0.00000 E0.38092:0.00000:50.00000
G1 X-25.00000 Y-25.00000 Z0.00000 E0.38092:0.00000:50.00000
G1 X25.00000 Y25.00000 Z0.00000 E0.38092:0.00000:50.00000
G1 X-25.00000 Y-25.00000 Z0.00000 E0.38092:0.00000:50.00000
G1 X25.00000 Y25.00000 Z0.00000 E0.38092:0.00000:50.00000
G1 X50.00000 Y0.00000 Z0.00000 E0.53870:0.00000:50.00000
G1 X0.00000 Y50.00000 Z0.00000 E0.53870:0.00000:50.00000
G1 X-50.00000 Y0.00000 Z0.00000 E0.53870:0.00000:50.00000
G1 X0.00000 Y-50.00000 Z0.00000 E0.53870:0.00000:50.00000
G0 X-10 Y-10 Z5
G90; Set to Absolute Positioning



visualize your code here: https://ncviewer.com/